# Simplex

Natasha Watkins

In [523]:
import numpy as np

## Problem 1 - 6

In [1274]:
class LinearProblem:
    
    def __init__(self, c, A, b):
        self.c, self.A, self.b = np.array(c), np.array(A), np.array(b)
        
        if np.any(self.b < 0):
            raise ValueError('Problem is infeasible')
        
        self.m, self.n = self.A.shape
        
        t = self.m + self.n
        self.L = np.concatenate([np.arange(self.n, t, 1),   # Basic variables
                                 np.arange(0, self.n, 1)])  # Non-basic variables
        
        self.T = self.init_tableau()
        
    def init_tableau(self):
        I_m = np.eye(self.m)
        A_bar = np.hstack([self.A, I_m])
        c_bar = np.hstack([self.c, np.zeros(self.m)])
        T1 = np.hstack([0, -c_bar.T, 1])
        T2 = np.column_stack([self.b, A_bar, np.zeros(self.m)])
        return np.vstack([T1, T2])
    
    def pivot_id(self):
        T = self.T.copy()
        col_id = int(np.argwhere(T[0, 1:] < 0)[0]) + 1
                
        if np.all(T[:, col_id] <= 0):
            raise ValueError('Problem is unbounded')
            
        nonpos_T = T[:, col_id] <= 0
        T[nonpos_T, col_id] = np.nan
        row_id = np.nanargmin(T[:, 0] / T[:, col_id])
                
        return row_id, col_id 
    
    def pivot(self):
                
        L, T, m = self.L, self.T, self.m
        row_id, col_id = self.pivot_id()
        
        L_temp = L.copy()
        
        L[row_id - 1] = L_temp[col_id + m - 1]
        L[col_id + m - 1] = L_temp[row_id - 1]
                        
        T[row_id, :] /= T[row_id, col_id]  # Divide pivot row
        
        for i in range(T.shape[0]):
            if i == row_id:
                pass
            else:
                k = -T[i, col_id]
                T[i, :] += k * T[row_id, :]
        
        self.T = T
        self.L = L
        
    def solve(self):
        
        while np.any(self.T[0, :] < 0):
            self.pivot()
                    
        max_value = self.T[0, 0]
        basic = self.L[:self.m]
        nonbasic = self.L[self.m:]
        d1 = dict(zip(nonbasic, np.zeros(self.m)))
        d2 = dict(zip(basic, self.T[1:, 0]))
        return (max_value, d1, d2)      

In [1275]:
c = [3, 2]
A = [[1, -1], 
     [3, 1],
     [4, 3]]
b = [2, 5, 7]

In [1276]:
lp = LinearProblem(c, A, b)
lp.solve()

(5.2,
 {3: 0.0, 4: 0.0},
 {0: 1.6, 1: 0.19999999999999982, 2: 0.5999999999999996})

## Problem 7

In [1277]:
data = np.load('productMix.npz')
a = data['A']
p = data['p']
m = data['m']
d = data['d']

In [1279]:
A = np.row_stack([a, np.eye((4))])
b = np.concatenate([m, d])

In [1280]:
LinearProblem(p, A, b).solve()

(7453.596491228071,
 {4: 0.0, 5: 0.0, 7: 0.0, 9: 0.0},
 {0: 10.0,
  1: 6.192982456140348,
  2: 12.0,
  3: 1.7894736842105292,
  6: 0.9659649122807016,
  8: 13.807017543859653,
  10: 8.21052631578947})